In [1]:
import os

In [2]:
def save_data(data, data_type):
    # Lưu dữ liệu đã được tiền xử lý
    file_path = os.path.join("data", data_type + ".json")  # Đường dẫn đến tệp JSON
    with open(file_path, "w") as f:
        json.dump(data, f, indent=4)
    return file_path

In [3]:
import json

with open("train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)
f.close()

from sklearn.model_selection import train_test_split
train_data, valid_data  = train_test_split(train_data, test_size= 0.1)

In [5]:
!mkdir data
train_data =  train_data
valid_data =  valid_data

save_data(train_data, "new_train")
save_data(valid_data, "valid")

'data/valid.json'

# **Load data & Train model**

In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


1. Tải dữ liệu

In [8]:
from datasets import load_dataset
raw_datasets = load_dataset("visquad_1.py")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:929: FutureWarning: The repository for visquad_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at visquad_1.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [9]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Cộng hòa Congo (tiếng Pháp: République du Congo), còn được gọi là Congo, Congo Cộng hòa, Tây Congo [cần dẫn nguồn], hoặc Congo-Brazzaville, là một quốc gia nằm trong Trung Phi. Nó được bao bọc bởi năm quốc gia: Gabon về phía tây; Cameroon ở phía tây bắc; the Cộng hòa Trung Phi ở phía đông bắc; Cộng hòa Dân chủ Congo ở phía đông và phía nam; và exclave Angola của Cabinda về phía tây nam. .
Question:  các thuật ngữ tiếng Pháp cho Cộng hòa Congo là gì?
Answer:  {'text': ['République du Congo),'], 'answer_start': [28]}


**Comment**

*   Trường context chứa văn bản mô tả bối cảnh của câu hỏi
*   Trường question chứa câu hỏi được đặt ra dựa trên ngữ cảnh
* Trường answer chứa câu trả lời cho câu hỏi

2. Lọc các mẫu dữ liệu

In [11]:
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) == 1)

Filter:   0%|          | 0/44954 [00:00<?, ? examples/s]

In [12]:
raw_datasets["validation"] = raw_datasets["validation"].filter(lambda x: len(x["answers"]["text"]) == 1)

Filter:   0%|          | 0/5092 [00:00<?, ? examples/s]

In [13]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'is_impossible'],
        num_rows: 44954
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'is_impossible'],
        num_rows: 5092
    })
})

3. Phân tách văn bản ngữ cảnh và câu hỏi

In [14]:
!pip install transformers

In [15]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [16]:
import torch
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("vinai/bartpho-syllable")
tokenizer.is_fast

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartphoTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


True

In [17]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s> các thuật ngữ tiếng Pháp cho Cộng hòa Congo là gì?</s></s> Cộng hòa Congo (tiếng Pháp: République du Congo), còn được gọi là Congo, Congo Cộng hòa, Tây Congo [cần dẫn nguồn], hoặc Congo-Brazzaville, là một quốc gia nằm trong Trung Phi. Nó được bao bọc bởi năm quốc gia: Gabon về phía tây; Cameroon ở phía tây bắc; the Cộng hòa Trung Phi ở phía đông bắc; Cộng hòa Dân chủ Congo ở phía đông và phía nam; và exclave Angola của Cabinda về phía tây nam..</s>'

4. Xử lý chiều dài câu

In [18]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))


<s> các thuật ngữ tiếng Pháp cho Cộng hòa Congo là gì?</s></s> Cộng hòa Congo (tiếng Pháp: République du Congo), còn được gọi là Congo, Congo Cộng hòa, Tây Congo [cần dẫn nguồn], hoặc Congo-Brazzaville, là một quốc gia nằm trong Trung Phi. Nó được bao bọc bởi năm quốc gia: Gabon về phía tây; Cameroon ở phía tây bắc; the Cộng hòa Trung Phi ở phía đông</s>
<s> các thuật ngữ tiếng Pháp cho Cộng hòa Congo là gì?</s></s>, hoặc Congo-Brazzaville, là một quốc gia nằm trong Trung Phi. Nó được bao bọc bởi năm quốc gia: Gabon về phía tây; Cameroon ở phía tây bắc; the Cộng hòa Trung Phi ở phía đông bắc; Cộng hòa Dân chủ Congo ở phía đông và phía nam; và exclave Angola của Cabinda về phía tây nam..</s>


In [19]:
a = " hiện đại đều là phân nhánh Lissamphibia của nhóm lớn Amphibia này. Động vật lưỡng cư phải trải qua quá trình biến thái từ ấu trùng sống dưới nước tới dạng trưởng thành có phổi thở không khí,"
c = tokenizer.tokenize(a)
print(len(c))

50


In [20]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=False,
)
print(inputs.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'])


5. Xác định vị trí câu trả lời

In [21]:
inputs["overflow_to_sample_mapping"]

[0, 0]

In [22]:
inputs = tokenizer(
    raw_datasets["train"][2:5]["question"],
    raw_datasets["train"][2:5]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 3 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 3 examples gave 8 features.
Here is where each comes from: [0, 0, 1, 1, 2, 2, 2, 2].


In [23]:
raw_datasets["train"][2:5]["answers"]

[{'text': ['Cameroon'], 'answer_start': [230]},
 {'text': ['Cộng hòa Trung Phi'], 'answer_start': [259]},
 {'text': ['cơn bão gây ra một mối đe dọa nghiêm trọng'], 'answer_start': [0]}]

In [24]:
answers = raw_datasets["train"][2:5]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer["answer_start"][0]
    end_char = answer["answer_start"][0] + len(answer["text"][0])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([81, 49, 92, 60, 18, 0, 0, 0], [83, 51, 96, 64, 30, 0, 0, 0])

In [25]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Cameroon, labels give: Cameroon


In [26]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx]["text"][0]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: cơn bão gây ra một mối đe dọa nghiêm trọng, decoded example: <s> Có gì sở hữu một mối đe dọa mỗi năm để Florida</s></s> cơn bão gây ra một mối đe dọa nghiêm trọng mỗi năm trong suốt 01 Tháng Sáu đến mùa bão ngày 30 tháng 11, đặc biệt là từ tháng Tám đến tháng Mười. Florida là tiểu bang có bão nhất, với nước nhiệt đới hoặc cận nhiệt đới trên bờ biển dài. Trong số loại 4 hoặc cao hơn bão đã tấn công Hoa Kỳ, 83% đã hoặc</s>


6. Tạo bộ dữ liệu huấn luyện và đánh giá

In [27]:
max_length = 1024
stride = 128
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
def preprocess_training_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
         max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [28]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_dataset,
    batched=True,
    batch_size=20,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/44954 [00:00<?, ? examples/s]

In [29]:
def preprocess_validation_dataset(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [31]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_dataset,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/5092 [00:00<?, ? examples/s]

(5092, 5092)

In [32]:
!pip install evaluate

7. Định nghĩa hàm đánh giá

In [33]:
from tqdm.auto import tqdm
import evaluate
import numpy as np
import collections

n_best = 20
max_answer_length = 30
predicted_answers = []
metric = evaluate.load("squad")

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

# **Default Training**

8. Tạo model

In [34]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [35]:
!pip install transformers[torch]

In [36]:
!pip install transformers

In [37]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("vinai/bartpho-syllable")

epochs = 10
batch_size = 4
lr = 2e-5

training_args = TrainingArguments(
    output_dir="checkpoints",
    evaluation_strategy="steps",  # print evaluation after finishing an epoch
    num_train_epochs=epochs,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=1,
    save_steps=2000,
    eval_steps=2000,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Some weights of MBartForQuestionAnswering were not initialized from the model checkpoint at vinai/bartpho-syllable and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [38]:
!pip install transformers -U

# **Custom training**

9. Tạo pipeline

In [39]:
!pip install transformers

from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "bert-base-uncased"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = "Năm 1871, Đức trở thành một quốc gia dân tộc khi hầu hết các quốc gia Đức thống nhất trong Đế quốc Đức do Phổ chi phối. Sau Chiến tranh thế giới thứ nhất và Cách mạng Đức 1918-1919, Đế quốc này bị thay thế bằng Cộng hòa Weimar theo chế độ nghị viện. Chế độ độc tài quốc xã được hình thành vào năm 1933, dẫn tới Chiến tranh thế giới thứ hai và một nạn diệt chủng. Sau một giai đoạn Đồng Minh chiếm đóng, hai nước Đức được thành lập: Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức. Năm 1990, quốc gia được tái thống nhất."

question = "Cộng hòa Weimar chính thức thay thế đế quốc Đức kể từ sau sự kiện nào?"
result = question_answerer(question=question, context=context)

print(f"Question: {question}")
print(f"Answer: {result['answer']}")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Question: Cộng hòa Weimar chính thức thay thế đế quốc Đức kể từ sau sự kiện nào?
Answer: vào năm 1933, dẫn tới Chiến tranh
